In [1]:
import os
import requests

import pandas as pd
import openai
import tiktoken

api_key = 'key'


In [13]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613"):
    """Return the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        print("Warning: model not found. Using cl100k_base encoding.")
        encoding = tiktoken.get_encoding("cl100k_base")
    if model in {
        "gpt-3.5-turbo-0613",
        "gpt-3.5-turbo-16k-0613",
        "gpt-4-0314",
        "gpt-4-32k-0314",
        "gpt-4-0613",
        "gpt-4-32k-0613",
        }:
        tokens_per_message = 3
        tokens_per_name = 1
    elif model == "gpt-3.5-turbo-0301":
        tokens_per_message = 4  # every message follows <|start|>{role/name}\n{content}<|end|>\n
        tokens_per_name = -1  # if there's a name, the role is omitted
    elif "gpt-3.5-turbo" in model:
        print("Warning: gpt-3.5-turbo may update over time. Returning num tokens assuming gpt-3.5-turbo-0613.")
        return num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613")
    elif "gpt-4" in model:
        print("Warning: gpt-4 may update over time. Returning num tokens assuming gpt-4-0613.")
        return num_tokens_from_messages(messages, model="gpt-4-0613")
    else:
        raise NotImplementedError(
            f"""num_tokens_from_messages() is not implemented for model {model}. See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens."""
        )
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>
    return num_tokens



In [3]:
reviews = pd.read_parquet('D:/Disco/Data/datasets/amazon_us_reviews/reviews_aws.parquet')

In [4]:
reviews[['review_body']][:10]

reviews[(reviews['product_category'] == 'Watches') & (reviews['product_title']== 'Timex Women\'s Easy Reader Leather Strap Watch')][['review_body']][:10]
product = reviews[(reviews['product_category'] == 'Watches') & (reviews['product_title']== 'Timex Women\'s Easy Reader Leather Strap Watch')]

In [12]:
prompt = 'Vou informar uma lista de comentarios, preciso que todos os cometarios sejam resumidos \
 em um unico comentario para que assim seja possível capturar o que os clientes estão pensando do produto, \
    seja aspectos positivos ou negativos e também destacando os pontos positivos e negativos do produto. \
        é necessário que o resumo geral seja escrito no estilo de um comentário de um cliente.'

In [14]:
openai.api_key = api_key

#"gpt-3.5-turbo"
chat_completion = openai.ChatCompletion.create(
    model="gpt-4", 
    messages=[{
        "role": "user",
        "content": f"{prompt}:{product['review_body']}"}])

In [15]:
chat_completion["choices"][0]["message"]["content"]

'Comprei este relógio e estou adorando. O aspecto positivo para a maioria de nós é o mostrador grande e bem iluminado à noite, que é bem fácil de ler. Além disso, o relógio parece ótimo e a banda, embora seja um pouco rígida, está ficando mais macia à medida que a uso. Além disso, muitos de nós adoramos as variedades de cores e materiais que a Timex oferece. Dito isso, existem alguns aspectos negativos. Por exemplo, o relógio não tem altímetro, pedômetro, GPS ou qualquer função digital. Outros pontos negativos incluem o fato de o relógio parar de funcionar temporariamente quando molhado, o som alto dos segundos correndo pode ser irritante e, apesar de ser fácil de ler, a luz indigo é notavelmente fraca. Finalmente, descobri que o relógio está atrasando um pouco. Tirando essas coisas, para nós que não precisamos de nada muito sofisticado, o relógio é uma ótima escolha e a maioria de nós está bastante satisfeita!'

In [57]:
product[['review_body']].to_csv('D:/Disco/Data/datasets/amazon_us_reviews/reviews_aws_product.csv', index=False)

In [11]:
product['review_body']

8145                                             Love it!!!
28105                                                Nice !
33420     No altimeter, no pedometer, no digital, no GPS...
35676     I love this watch. The led light at nite is gr...
41837         It's a gift for my mother,the watch is good !
42117     Love the oversize face!  The band is a bit sti...
49303     I bought this for my Mother who is 86. It is e...
50364     timex has always been good quality and afforda...
50934     I need a larger face in order to read the time...
57063     love this watch, It&#34;s large enough to see ...
63760     Just what I wanted, clear face, glows in the d...
69063     Easy to read and looks great, but runs slow. W...
70327     Few watches have a white face and black hands....
71051     This was bought as a Christmas gift. The recip...
82340     I like these watches and wear one all the time...
82523     Purchased as a gift for my elderly Mom.  She l...
93599     Just received this today-it's 

In [16]:
len(reviews)

12645269

In [ ]:
Gerar uma review geral de um produto a partir de informações existentes nas reviews realizadas pelos clientes que adiquiriram o produto em questão.